# Import Libraries
This code runs on <b>Julia 1.8.5</b> with the following packages and versions:

- BSON: 0.3.6
- UMAP: 0.1.9
- OrderedCollections: 1.4.1
- DataFrames: 1.5.0
- ScikitLearn: 0.6.6
- Formatting: 0.4.2
- Gadfly: 1.3.4
- PyCall: 1.95.1
- StatsBase: 0.33.21
- ShapML: 0.3.2
- HDF5: 0.16.13 
- IJulia: 1.24.0
- Flux: 0.13.12
- Plots: 1.38.5
- PlotlyJS: 0.18.10
- ProgressMeter: 1.7.2
- XGBoost: 2.2.4

In [1]:
using HDF5, Plots, ProgressMeter, OrderedCollections, Statistics, ScikitLearn, Printf, Formatting
using Flux, CUDA, XGBoost, StatsBase, ShapML, DataFrames, Gadfly, UMAP, PyCall, PlotlyJS
using BSON: @load, @save


@sk_import preprocessing: (StandardScaler, MinMaxScaler)
@sk_import metrics: (roc_curve, auc, r2_score, classification_report, precision_recall_curve, average_precision_score)
@sk_import svm: (SVC, OneClassSVM, SVR)
@sk_import model_selection: train_test_split
@sk_import discriminant_analysis: LinearDiscriminantAnalysis
@sk_import neighbors: LocalOutlierFactor
@sk_import ensemble: (RandomForestClassifier, RandomForestRegressor, ExtraTreesClassifier, ExtraTreesRegressor)
@sk_import decomposition: PCA
@sk_import neural_network: (MLPClassifier, MLPRegressor)

plotlyjs()

WebIO._IJuliaInit()

[ Info: Running `conda install -y -c conda-forge llvm-openmp` in root environment


Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.4.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.4.0


[ Info: Running `conda install -y -c conda-forge llvm-openmp` in root environment


Solving environment: ...working... done

# All requested packages already installed.



...

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.4.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.4.0


[ Info: Running `conda install -y -c conda-forge llvm-openmp` in root environment


Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.4.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.4.0




Plots.PlotlyJSBackend()

# Starter Functions
These functions will help pre-process the dataset by loading all subsets and extracting (time domain summary statistic) features from each. We use the built-in NASA split, which divides the engines with a training-testing split of 60%-40%. Lastly, we further divide the training set into a random 80%-20% training-validation split.

In [2]:
function get_Data()
    filename_list = ["../Data/N-CMAPSS_DS01-005.h5",
    "../Data/N-CMAPSS_DS03-012.h5",
    "../Data/N-CMAPSS_DS04.h5",
    "../Data/N-CMAPSS_DS05.h5",
    "../Data/N-CMAPSS_DS06.h5",
    "../Data/N-CMAPSS_DS07.h5",
    "../Data/N-CMAPSS_DS08a-009.h5",
    "../Data/N-CMAPSS_DS08c-008.h5"]

    dataset_list = ["DS01", "DS03", "DS04", "DS05", "DS06", "DS07", "DS08a", "DS08c"]
    failure_modes = [1, 2, 3, 4, 5, 6, 7, 7]
    
    PHM_2021_Dataset_Dict = OrderedDict()
    @showprogress for (i, filename) in enumerate(filename_list)
        fid = h5open(filename, "r")
        dev_dict, test_dict = separate_Units(fid);
        close(fid)
        PHM_2021_Dataset_Dict[dataset_list[i]] = Dict("Dev" => dev_dict, "Test" => test_dict, "Failure" => failure_modes[i])
    end
    return PHM_2021_Dataset_Dict
end

function separate_Units(fid)
## Inputs: Opened HDF5 file
## Outputs: Development dataset and test dataset separated by each unit + cycle
    A_dev = Array(fid["A_dev"]) # 4 rows: unit #, cycle #, flight class, health state 
    T_dev = Array(fid["T_dev"]) # 10 rows: failure mode info 
    W_dev = Array(fid["W_dev"]) # 4 rows: altitude, mach no., throttle resolver angle, T2 inlet temp
    X_s_dev = Array(fid["X_s_dev"]) # 14 rows: sensor measurements
    X_v_dev = Array(fid["X_v_dev"]) # 14 rows: virtual sensor measurements
    Y_dev = Array(fid["Y_dev"]) # 1 row: RUL labels

    dev_mat = (A_dev, T_dev, W_dev, X_s_dev, X_v_dev, Y_dev)

    A_test = Array(fid["A_test"])
    T_test = Array(fid["T_test"])
    W_test = Array(fid["W_test"])
    X_s_test = Array(fid["X_s_test"])
    X_v_test = Array(fid["X_v_test"])
    Y_test = Array(fid["Y_test"])
    
    test_mat = (A_test, T_test, W_test, X_s_test, X_v_test, Y_test)
    
    return dev_mat, test_mat
end

function explain_Failures(failure)
    # Returns component-level failure vector based on the labeled failure modes from NASA
    # Order: [Fan, LPC, HPC, HPT, LPT]
    # Note that some failure modes involve simultaneous component failures! 
    if failure == 1
        fail_vec = [0; 0; 0; 1; 0] # DS01 HPT efficiency failure
    
    elseif failure == 2
        fail_vec = [0; 0; 0; 1; 1] # DS03 HPT efficiency, LPT efficiency + flow failures
    
    elseif failure == 3
        fail_vec = [1; 0; 0; 0; 0] # DS04 Fan efficiency + flow failure
    
    elseif failure == 4
        fail_vec = [0; 0; 1; 0; 0] # DS05 HPC efficiency + flow failure
    
    elseif failure == 5
        fail_vec = [0; 1; 1; 0; 0] # DS06 LPC/HPC efficiency + flow failures
    
    elseif failure == 6
        fail_vec = [0; 0; 0; 0; 1] # DS07 LPT efficiency + flow failures
    
    else
        fail_vec = [1; 1; 1; 1; 1] # DS08a/c all component efficiency + flow failures
    end

    return fail_vec
end

function extract_Features(PHM_2021_Dataset_Dict, mode)
    # Extracts summary statistics from each time series in all datasets
    # Uses NASA's train-test split with second input, mode ∈ ["Dev", "Test"]
    dataset_list = ["DS01", "DS03", "DS04", "DS05", "DS06", "DS07", "DS08a", "DS08c"]
    feature_mat = []
    label_vec = []
    @showprogress for dataset in dataset_list
        file_set = PHM_2021_Dataset_Dict[dataset]
        dataset = file_set[mode]
        A_dev = dataset[1]
        W_dev = dataset[3]
        X_s_dev = dataset[4]
        Y_dev = dataset[end]

        units = unique(A_dev[1, :])
        for unit in units

            unit_idxs = A_dev[1, :] .== unit
            unit_mat = A_dev[:, unit_idxs]

            cycles = unique(unit_mat[2, :])

            for cycle in cycles

                cycle_idxs = unit_mat[2, :] .== cycle

                A = unit_mat[:, cycle_idxs]
                W = W_dev[:, unit_idxs][:, cycle_idxs]
                X = X_s_dev[:, unit_idxs][:, cycle_idxs]
                Y = Y_dev[:, unit_idxs][:, cycle_idxs]

                feature_vec = []
                append!(feature_vec, [size(A,2); A[2,1]; A[3, 1]]) # total duration, cycle, flight class (consts per cycle)
                [append!(feature_vec, [mean(W[i,:]); std(W[i,:]); quantile(W[i,:], [0.0, 0.25, 0.50, 0.75, 1.0])]) for i in 1:size(W,1)] # W's: mean, std, min, 0.25th quantile, median, 0.75th quantile, max
                [append!(feature_vec, [mean(X[i,:]); std(X[i,:]); quantile(X[i,:], [0.0, 0.25, 0.50, 0.75, 1.0])]) for i in 1:size(X,1)] # X_s's: mean, std, min, 0.25th quantile, median, 0.75th quantile, max
                push!(feature_mat, feature_vec)
                push!(label_vec, Int(A[4,1])) # Current Health State (const per cycle)
                append!(label_vec, explain_Failures(file_set["Failure"])) # Eventual Failure Mode broken down into 5 components
                push!(label_vec, Y[1]) # RUL label (const per cycle)
            end
        end
    end
    
    feature_mat = vcat(transpose.(feature_mat)...)
    label_mat = Array(reshape(label_vec, (7, Int( length(label_vec) / 7))  )')
    return feature_mat, label_mat
end

function get_Lifetimes(labels)
    # Extracts total lifetimes for each cycle
    # Changes in vector represent a new engine unit
    lifetime_idxs = [1; findall(diff(labels[:, end]) .!= -1) .+ 1]
    unit_lifetimes = labels[lifetime_idxs, end]

    lifetimes = []
    for i in unit_lifetimes
        append!(lifetimes, i*ones(i+1))
    end
    return lifetimes
end

get_Lifetimes (generic function with 1 method)

In [3]:
PHM_2021_Dataset_Dict = get_Data()

Progress: 100%|█████████████████████████████████████████| Time: 0:00:09


OrderedDict{Any, Any} with 8 entries:
  "DS01"  => Dict{String, Any}("Dev"=>([1.0 1.0 … 6.0 6.0; 1.0 1.0 … 94.0 94.0;…
  "DS03"  => Dict{String, Any}("Dev"=>([1.0 1.0 … 9.0 9.0; 1.0 1.0 … 84.0 84.0;…
  "DS04"  => Dict{String, Any}("Dev"=>([1.0 1.0 … 6.0 6.0; 1.0 1.0 … 83.0 83.0;…
  "DS05"  => Dict{String, Any}("Dev"=>([1.0 1.0 … 6.0 6.0; 1.0 1.0 … 67.0 67.0;…
  "DS06"  => Dict{String, Any}("Dev"=>([1.0 1.0 … 6.0 6.0; 1.0 1.0 … 68.0 68.0;…
  "DS07"  => Dict{String, Any}("Dev"=>([1.0 1.0 … 6.0 6.0; 1.0 1.0 … 74.0 74.0;…
  "DS08a" => Dict{String, Any}("Dev"=>([1.0 1.0 … 9.0 9.0; 1.0 1.0 … 95.0 95.0;…
  "DS08c" => Dict{String, Any}("Dev"=>([1.0 1.0 … 6.0 6.0; 1.0 1.0 … 60.0 60.0;…

In [4]:
x_train, y_train = extract_Features(PHM_2021_Dataset_Dict, "Dev")
x_test, y_test = extract_Features(PHM_2021_Dataset_Dict, "Test")

lifetimes_train = get_Lifetimes(y_train) 
lifetimes_test = get_Lifetimes(y_test)

x_train, x_val, y_train, y_val, lifetimes_train, lifetimes_val = train_test_split(x_train, y_train, lifetimes_train, test_size=0.20, random_state=42)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:54
Progress: 100%|█████████████████████████████████████████| Time: 0:01:13


6-element Vector{Array}:
 [5130.0 92.0 … 3.1048813908840023 5.13283858704109; 5378.0 25.0 … 3.1562889873245874 5.12186906439488; … ; 16503.0 21.0 … 2.47226887249876 4.82710077948126; 3121.0 13.0 … 3.74647736945852 4.93512470052458]
 [4181.0 47.0 … 3.1102671733940697 5.2377986321574; 8227.0 55.0 … 3.224917946863645 5.1602269863800005; … ; 4181.0 38.0 … 3.1142786442598904 5.24640816203981; 7795.0 47.0 … 3.154163314355415 5.0340431149809595]
 [0 0 … 0 7; 1 0 … 0 62; … ; 0 1 … 1 37; 1 1 … 1 59]
 [0 0 … 1 25; 0 0 … 1 12; … ; 0 1 … 1 34; 0 0 … 0 53]
 [98.0, 86.0, 84.0, 99.0, 79.0, 99.0, 52.0, 70.0, 99.0, 77.0  …  66.0, 99.0, 73.0, 94.0, 70.0, 70.0, 86.0, 92.0, 57.0, 71.0]
 [71.0, 66.0, 98.0, 79.0, 86.0, 77.0, 72.0, 94.0, 74.0, 79.0  …  98.0, 70.0, 55.0, 77.0, 98.0, 99.0, 99.0, 66.0, 71.0, 99.0]

# Benchmark Approaches

Here, we compare the results of Random Forest, Extreme Random Forest, XGBoost, and an ANN using scikit-learn's implementations. The reg_Results function provides classification reports for each failure type using a set threshold; we also provide the AUROC and AUPR metrics for aggregate measures that are not specific to a single threshold. We also report additional metrics based on RUL including RMSE, MAE (as percentage of lifetime as well as cycles).

In [5]:
function reg_Results(model_name, y_preds, y_true, lifetimes)
    
    auc_list = []
    apr_list = []
    if model_name == "Flux"
        for i=1:6
            fpr, tpr, thresholds = roc_curve(y_true[:, i], sigmoid(y_preds[:, i]), pos_label=1)
            precision, recall, thresholds = precision_recall_curve(y_true[:,i], sigmoid(y_preds[:,i]), pos_label=1)
            push!(auc_list, auc(fpr, tpr))
            push!(apr_list, auc(recall, precision))        
        end
        rounded_preds = Array(sigmoid(y_preds[:, 1:end-1]) .>= .50)
        rounded_trues = Array(y_true[:, 1:end-1] .>= .50)
    else
        for i=1:6
            fpr, tpr, thresholds = roc_curve(y_true[:, i], y_preds[:, i], pos_label=100)
            precision, recall, thresholds = precision_recall_curve(y_true[:,i], y_preds[:,i], pos_label=100)
            push!(auc_list, auc(fpr, tpr))
            push!(apr_list, auc(recall, precision))        
        end
        rounded_preds = Array(y_preds[:, 1:end-1] .>= 50)
        rounded_trues = Array(y_true[:, 1:end-1] .>= 50)
    end
    
    y_hat = hcat(rounded_preds, y_preds[:, end])
    y_true = hcat(rounded_trues, y_true[:, end])
    
    print(classification_report(y_true[:, 1], y_hat[:, 1],  target_names = ["$model_name: Unhealthy", "$model_name: Healthy"]))
    print(classification_report(y_true[:, 2], y_hat[:, 2],  target_names = ["$model_name: No Fan Failure", "$model_name: Fan Failure"]))
    print(classification_report(y_true[:, 3], y_hat[:, 3],  target_names = ["$model_name: No LPC Failure", "$model_name: LPC Failure"]))
    print(classification_report(y_true[:, 4], y_hat[:, 4],  target_names = ["$model_name: No HPC Failure", "$model_name: HPC Failure"]))
    print(classification_report(y_true[:, 5], y_hat[:, 5],  target_names = ["$model_name: No HPT Failure", "$model_name: HPT Failure"]))
    print(classification_report(y_true[:, 6], y_hat[:, 6],  target_names = ["$model_name: No LPT Failure", "$model_name: LPT Failure"]))
    printfmt("AUROC Scores: {} \n", auc_list)
    printfmt("AUPR Scores: {} \n", apr_list)
    
    RUL_preds = y_preds[:, end]
    RUL_true = y_true[:, end]
    
    mean_RUL_error = mean(abs.(RUL_preds - RUL_true))
    RMSE_RUL_error = sqrt(mean( (RUL_preds - RUL_true) .^ 2))
    NASA_RUL_error = getNASAScore(RUL_preds, RUL_true)
    
    printfmt("Mean RUL Error: {:.3f} \n", mean_RUL_error)
    printfmt("RUL RMSE: {:.3f} \n", RMSE_RUL_error)
    printfmt("RUL NASA Score: {:.3f} \n", getNASAScore(RUL_preds, RUL_true))

    errors = RUL_preds - RUL_true
    abs_errors = abs.(errors)

    for percent in [0.20, 0.15, 0.10, 0.05]
        errors_within_percent = mean(abs_errors .<= (lifetimes .* percent))
        printfmt("{:.2f}% of RUL Predictions Within {:d}% of EOL \n", 100*errors_within_percent, 100*percent)
    end
    printfmt("Average Prediction Error (% of Lifetime) {:.2f}% \n", mean(100*(abs_errors ./ lifetimes)))
    printfmt("Std Prediction Error (% of Lifetime) {:.2f}% \n", std(100*(abs_errors ./ lifetimes)))
end

function supervised_Regression(x_train, x_test, y_train, y_test, lifetimes)
    std_scaler = MinMaxScaler()
    x_train_reduced = std_scaler.fit_transform(x_train)
    x_test_reduced = std_scaler.transform(x_test)
    
    #Put labels in "RUL scale" just for these models so regression can work!
    y_train = Float64.(hcat(y_train[:, 1:end-1] .* 100.0, y_train[:,end]))
    y_test = Float64.(hcat(y_test[:, 1:end-1] .* 100.0, y_test[:,end]))

    for model_name in ["RF", "ERF", "XGB", "ANN"]
        
        if model_name == "RF"
            rf_clf = RandomForestRegressor(n_estimators=100)
            rf_clf.fit(x_train_reduced, y_train)
            y_train_hat = rf_clf.predict(x_train_reduced)
            y_test_hat = rf_clf.predict(x_test_reduced)
            reg_Results(model_name, y_test_hat, y_test, lifetimes)
        end

        if model_name == "ERF"
            erf_clf = ExtraTreesRegressor(n_estimators=100)
            erf_clf.fit(x_train_reduced, y_train)
            y_train_hat = erf_clf.predict(x_train_reduced)
            y_test_hat = erf_clf.predict(x_test_reduced)
            reg_Results(model_name, y_test_hat, y_test, lifetimes)
        end

        if model_name == "XGB"
            pred_list = []
            n_estimators = 100
            for i = 1:size(y_train,2)
                bst = xgboost((x_train_reduced, y_train[:,i]), num_round=n_estimators, eta=0.3, max_depth=6)
                # predict
                push!(pred_list, XGBoost.predict(bst, x_test_reduced))
            end
            y_test_hat = hcat(pred_list...)
            reg_Results(model_name, y_test_hat, y_test, lifetimes)
        end
        
        if model_name == "ANN"
            ann_clf = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=5000)
            ann_clf.fit(x_train_reduced, y_train)
            y_train_hat = ann_clf.predict(x_train_reduced)
            y_test_hat = ann_clf.predict(x_test_reduced)
            reg_Results(model_name, y_test_hat, y_test, lifetimes)
        end
        
    end

end

function getNASAScore(y_pred, y_true)
    delta = y_pred - y_true #positive means overestimated for RUL
    rmse = sqrt( mean( delta .^ 2 ) )
    alpha = (1/13) .+ ((3/130) .* (delta .> 0))
    s_c = mean(exp.(alpha .* abs.(delta))) - 1
    return 0.5*rmse + 0.5*s_c
end

getNASAScore (generic function with 1 method)

In [6]:
since = time()

supervised_Regression(x_train, x_test, y_train, y_test, lifetimes_test)

elapsed_time = time() - since
printfmt("Time Elapsed: {:.2f} seconds", elapsed_time)

               precision    recall  f1-score   support

RF: Unhealthy       0.99      0.98      0.98      1865
  RF: Healthy       0.96      0.97      0.96       871

     accuracy                           0.98      2736
    macro avg       0.97      0.98      0.97      2736
 weighted avg       0.98      0.98      0.98      2736
                    precision    recall  f1-score   support

RF: No Fan Failure       0.75      0.86      0.80      1772
   RF: Fan Failure       0.65      0.48      0.55       964

          accuracy                           0.73      2736
         macro avg       0.70      0.67      0.68      2736
      weighted avg       0.72      0.73      0.72      2736
                    precision    recall  f1-score   support

RF: No LPC Failure       0.75      0.87      0.81      1794
   RF: LPC Failure       0.66      0.46      0.54       942

          accuracy                           0.73      2736
         macro avg       0.71      0.67      0.68      2736
    

[ Info: XGBoost: starting training.
[ Info: [1]	train-rmse:39.10733726836359381
[ Info: [2]	train-rmse:28.17406666103219948
[ Info: [3]	train-rmse:20.50596029106220186
[ Info: [4]	train-rmse:15.32082525974806408
[ Info: [5]	train-rmse:11.83107031198610670
[ Info: [6]	train-rmse:9.52378684473969983
[ Info: [7]	train-rmse:7.91908358836334259
[ Info: [8]	train-rmse:6.93487460409844747
[ Info: [9]	train-rmse:6.24383278451176071
[ Info: [10]	train-rmse:5.68922061134411194
[ Info: [11]	train-rmse:5.38665816895388172
[ Info: [12]	train-rmse:5.17650941893847527
[ Info: [13]	train-rmse:4.95813397633252695
[ Info: [14]	train-rmse:4.80698049108919800
[ Info: [15]	train-rmse:4.65529271820024171
[ Info: [16]	train-rmse:4.55642003370843085
[ Info: [17]	train-rmse:4.47765548726177975
[ Info: [18]	train-rmse:4.31448592699230193
[ Info: [19]	train-rmse:4.18000236521989965
[ Info: [20]	train-rmse:4.13685199111393676
[ Info: [21]	train-rmse:4.04737415753647678
[ Info: [22]	train-rmse:4.01068829906644719


[ Info: [2]	train-rmse:45.42282188592501058
[ Info: [3]	train-rmse:42.07148506785268438
[ Info: [4]	train-rmse:39.56838023868377974
[ Info: [5]	train-rmse:37.59734414948659520
[ Info: [6]	train-rmse:36.51343828287713222
[ Info: [7]	train-rmse:34.41089231861575115
[ Info: [8]	train-rmse:34.10290788773996695
[ Info: [9]	train-rmse:32.66358651629769838
[ Info: [10]	train-rmse:32.15014831689656916
[ Info: [11]	train-rmse:31.75376855611757421
[ Info: [12]	train-rmse:31.19586672830919838
[ Info: [13]	train-rmse:31.06928443910190651
[ Info: [14]	train-rmse:30.87757429318796554
[ Info: [15]	train-rmse:30.74291087502624364
[ Info: [16]	train-rmse:30.18552685902767507
[ Info: [17]	train-rmse:29.49898269075358570
[ Info: [18]	train-rmse:28.75678064081037633
[ Info: [19]	train-rmse:28.49686134999418741
[ Info: [20]	train-rmse:27.71433952830168224
[ Info: [21]	train-rmse:27.32362401826983245
[ Info: [22]	train-rmse:26.89595858761146374
[ Info: [23]	train-rmse:26.48145175626155634
[ Info: [24]	train

[ Info: [2]	train-rmse:45.21232185857861197
[ Info: [3]	train-rmse:42.01222595478633082
[ Info: [4]	train-rmse:39.71542124966217813
[ Info: [5]	train-rmse:38.31067964353743349
[ Info: [6]	train-rmse:37.71416568372773526
[ Info: [7]	train-rmse:35.72456086339864356
[ Info: [8]	train-rmse:34.50122724506714889
[ Info: [9]	train-rmse:33.91089702771090231
[ Info: [10]	train-rmse:32.86817012942056948
[ Info: [11]	train-rmse:32.25857805776191611
[ Info: [12]	train-rmse:31.96914716236982912
[ Info: [13]	train-rmse:31.41953049552495614
[ Info: [14]	train-rmse:30.57899677002221850
[ Info: [15]	train-rmse:29.44054151043513556
[ Info: [16]	train-rmse:28.50232902759770681
[ Info: [17]	train-rmse:28.31529850495340739
[ Info: [18]	train-rmse:28.09484382804113523
[ Info: [19]	train-rmse:27.89077244974508929
[ Info: [20]	train-rmse:27.12634151310733444
[ Info: [21]	train-rmse:26.44683917551851593
[ Info: [22]	train-rmse:26.26669186785793642
[ Info: [23]	train-rmse:25.60478577201826766
[ Info: [24]	train

[ Info: [4]	train-rmse:39.88108300874538514
[ Info: [5]	train-rmse:36.33229675341157616
[ Info: [6]	train-rmse:33.83353004997319857
[ Info: [7]	train-rmse:32.33109330750149013
[ Info: [8]	train-rmse:31.55004941302922106
[ Info: [9]	train-rmse:29.93346906498123872
[ Info: [10]	train-rmse:29.17571605991712147
[ Info: [11]	train-rmse:28.31993661516137806
[ Info: [12]	train-rmse:26.82955997750114108
[ Info: [13]	train-rmse:26.19890695032101391
[ Info: [14]	train-rmse:25.47940679532961283
[ Info: [15]	train-rmse:25.10394036548266428
[ Info: [16]	train-rmse:24.81757372291438912
[ Info: [17]	train-rmse:24.57726078712321538
[ Info: [18]	train-rmse:24.48574961868165900
[ Info: [19]	train-rmse:23.86902360609578722
[ Info: [20]	train-rmse:23.68141123739481557
[ Info: [21]	train-rmse:23.50070261319359233
[ Info: [22]	train-rmse:22.80547176484930105
[ Info: [23]	train-rmse:22.15610002520267230
[ Info: [24]	train-rmse:21.48255047134700035
[ Info: [25]	train-rmse:21.17501544774461664
[ Info: [26]	tra

[ Info: [6]	train-rmse:33.36090698283499023
[ Info: [7]	train-rmse:31.45441218718436716
[ Info: [8]	train-rmse:29.49471660646675275
[ Info: [9]	train-rmse:28.35727723579348947
[ Info: [10]	train-rmse:27.55884162497376977
[ Info: [11]	train-rmse:26.12420725684470213
[ Info: [12]	train-rmse:25.53426864698490562
[ Info: [13]	train-rmse:24.81814329886368142
[ Info: [14]	train-rmse:24.06689650966856320
[ Info: [15]	train-rmse:23.38864315172028796
[ Info: [16]	train-rmse:23.04386007939858416
[ Info: [17]	train-rmse:22.46424716570483326
[ Info: [18]	train-rmse:21.81331948893758366
[ Info: [19]	train-rmse:21.46103308105990592
[ Info: [20]	train-rmse:20.89309523738125662
[ Info: [21]	train-rmse:20.07268813487299752
[ Info: [22]	train-rmse:19.48290174864131785
[ Info: [23]	train-rmse:19.13854151040587581
[ Info: [24]	train-rmse:18.88042298674568897
[ Info: [25]	train-rmse:18.45610707370235204
[ Info: [26]	train-rmse:18.03698353945246780
[ Info: [27]	train-rmse:17.84370512466464120
[ Info: [28]	t

[ Info: [4]	train-rmse:39.64347466903752348
[ Info: [5]	train-rmse:37.25384723353507610
[ Info: [6]	train-rmse:35.05506279403684289
[ Info: [7]	train-rmse:33.50995368066008950
[ Info: [8]	train-rmse:32.51891118005558923
[ Info: [9]	train-rmse:31.07301522850029230
[ Info: [10]	train-rmse:30.43901743505677970
[ Info: [11]	train-rmse:29.44352884764910527
[ Info: [12]	train-rmse:28.91524360511004232
[ Info: [13]	train-rmse:28.25028185554019089
[ Info: [14]	train-rmse:27.53963797347828901
[ Info: [15]	train-rmse:27.21704518752857282
[ Info: [16]	train-rmse:27.03123824599054004
[ Info: [17]	train-rmse:26.81982553679412362
[ Info: [18]	train-rmse:26.19750658145178335
[ Info: [19]	train-rmse:25.86912909530599691
[ Info: [20]	train-rmse:25.25920633211746491
[ Info: [21]	train-rmse:24.80008125215443826
[ Info: [22]	train-rmse:24.57496021748491088
[ Info: [23]	train-rmse:24.11398045059474526
[ Info: [24]	train-rmse:23.25062379794565004
[ Info: [25]	train-rmse:22.71520135632305326
[ Info: [26]	tra

[ Info: [4]	train-rmse:14.48900192057565839
[ Info: [5]	train-rmse:11.91218531476565445
[ Info: [6]	train-rmse:10.31228015609313431
[ Info: [7]	train-rmse:9.28272580984145179
[ Info: [8]	train-rmse:8.53543291851280550
[ Info: [9]	train-rmse:7.91894929298934613
[ Info: [10]	train-rmse:7.56466043885256934
[ Info: [11]	train-rmse:7.33084940235317539
[ Info: [12]	train-rmse:7.01464744778559091
[ Info: [13]	train-rmse:6.76078615276193151
[ Info: [14]	train-rmse:6.68535947354611082
[ Info: [15]	train-rmse:6.49597122721960929
[ Info: [16]	train-rmse:6.45152035274352720
[ Info: [17]	train-rmse:6.26236362049911843
[ Info: [18]	train-rmse:6.21829510863921797
[ Info: [19]	train-rmse:6.12830259792591914
[ Info: [20]	train-rmse:6.04712711755433663
[ Info: [21]	train-rmse:5.96190103270423233
[ Info: [22]	train-rmse:5.88207173311945919
[ Info: [23]	train-rmse:5.85257925306466120
[ Info: [24]	train-rmse:5.82069576750397477
[ Info: [25]	train-rmse:5.71739221245823082
[ Info: [26]	train-rmse:5.694874990

                precision    recall  f1-score   support

XGB: Unhealthy       0.98      0.98      0.98      1865
  XGB: Healthy       0.95      0.97      0.96       871

      accuracy                           0.97      2736
     macro avg       0.97      0.97      0.97      2736
  weighted avg       0.97      0.97      0.97      2736
                     precision    recall  f1-score   support

XGB: No Fan Failure       0.82      0.90      0.86      1772
   XGB: Fan Failure       0.77      0.63      0.70       964

           accuracy                           0.81      2736
          macro avg       0.80      0.77      0.78      2736
       weighted avg       0.80      0.81      0.80      2736
                     precision    recall  f1-score   support

XGB: No LPC Failure       0.84      0.90      0.87      1794
   XGB: LPC Failure       0.79      0.68      0.73       942

           accuracy                           0.82      2736
          macro avg       0.81      0.79      0.

# Training the xANN Model with Flux
We include the code for training the xANN model with Flux, with two settings for the loss function ("MSE" and "Flux"). The MSE setting involves a rudimentary scaling of the binary labels (multiplying by 100 to have them roughly weigh as much as the RUL). The "Flux" setting attempts to penalize overestimates by applying NASA's scoring function.

In [7]:
function Flux_DNN(x_train, x_test, y_train, y_test, lifetimes, loss_fn)
    num_inputs = size(x_train, 2)
    num_outputs = size(y_train, 2)
    model = create_Model(num_inputs, num_outputs) 
    
    std_scaler = MinMaxScaler()
    x_train_reduced = std_scaler.fit_transform(x_train)
    x_test_reduced = std_scaler.transform(x_test)
    
    model_name = loss_fn
    
    if loss_fn == "MSE"
        y_train = Float64.(hcat(y_train[:, 1:end-1] .* 100.0, y_train[:,end]))
        y_test = Float64.(hcat(y_test[:, 1:end-1] .* 100.0, y_test[:,end]))
        loss = (x,y)->Flux.Losses.mse(model(x), y)
    end
    
    if loss_fn == "Flux"
        loss = (x,y)->begin 
            delta = model(x) - y #positive means overestimated for RUL
            rmse = sqrt( mean( delta[end, :] .^ 2 ) )
            alpha = (1/13) .+ ((3/130) .* (delta[end, :] .> 0))
            s_c = mean(exp.(alpha .* abs.(delta[end, :]))) - 1
            return 0.5*rmse + 0.5*s_c + 10*Flux.logitbinarycrossentropy(model(x)[1:end-1, :], y[1:end-1, :])
        end        
    end
    
    x_train = Array(x_train_reduced')
    x_test = Array(x_test_reduced') 
    y_train = Array(y_train') 
    y_test = Array(y_test') 
        
    ps = Flux.params(model)
    data = Flux.DataLoader((x_train, y_train), batchsize=256) 
    evalcb() = @show(loss(x_test, y_test) )
    throttled_cb = Flux.throttle(evalcb, 5)
    opt = Flux.ADAM(0.001, (0.9, 0.999))
    epochs = 5000
    
    best_params = ps
    best_loss = Inf
    @showprogress for i = 1:epochs
        Flux.train!(loss, ps, data, opt, cb=throttled_cb)
        test_loss = loss(x_test, y_test)

        if test_loss < best_loss
            best_loss = test_loss
            best_model = cpu(model)
       #     @save "../Models/xANN_Parameters.bson" best_model
        end
        
     end
    printfmt("Best Loss: {:.4f}\n\n", best_loss)
    y_test_hat = Array(model(x_test)')
    reg_Results(model_name, y_test_hat, Array(y_test'), lifetimes)
    return model
end

function create_Model(num_inputs, num_outputs)
    
    model = Chain(
        Dense(num_inputs, 64, relu),
        Dense(64, 32, relu),
        Dense(32, num_outputs))
    
    return model
end

create_Model (generic function with 1 method)

When we call Flux_DNN, we pass in the validation set and report the same metrics as above on the validation set. In the next step, we will evaluate performance on the testing set (evaluating generalization performance on unseen engine units).

In [8]:
since = time()
loss_fn = "MSE" # loss_fn ∈ ["MSE", "Flux"] 
flux_model = Flux_DNN(x_train, x_val, y_train, y_val, lifetimes_val, loss_fn)

elapsed_time = time() - since
printfmt("Time Elapsed: {:.2f} seconds", elapsed_time)

loss(x_test, y_test) = 3798.2407011071236


Progress:   5%|██                                       |  ETA: 0:04:39

loss(x_test, y_test) = 1528.8968337979697


Progress:   9%|███▉                                     |  ETA: 0:03:05

loss(x_test, y_test) = 1322.5353768014922


...

loss(x_test, y_test) = 665.1283598194706


Progress:  95%|██████████████████████████████████████▉  |  ETA: 0:00:05

loss(x_test, y_test) = 650.2803699502068


Progress: 100%|█████████████████████████████████████████| Time: 0:01:28


Best Loss: 653.1673

                precision    recall  f1-score   support

MSE: Unhealthy       0.98      0.98      0.98       570
  MSE: Healthy       0.96      0.96      0.96       248

      accuracy                           0.98       818
     macro avg       0.97      0.97      0.97       818
  weighted avg       0.98      0.98      0.98       818
                     precision    recall  f1-score   support

MSE: No Fan Failure       0.95      0.97      0.96       543
   MSE: Fan Failure       0.93      0.89      0.91       275

           accuracy                           0.94       818
          macro avg       0.94      0.93      0.93       818
       weighted avg       0.94      0.94      0.94       818
                     precision    recall  f1-score   support

MSE: No LPC Failure       0.91      0.87      0.89       545
   MSE: LPC Failure       0.77      0.82      0.79       273

           accuracy                           0.86       818
          macro avg       0

# Visualizing xANN Predictions

We load xANN parameters using the provided .bson file. These parameters were obtained in the MSE setting, but we encourage users to train the xANN parameters differently (i.e. in the "Flux" setting, or with different hyperparameters) and load those parameters. First, we show results on the test set, and provide some visuals for component-level prediction performance. Then, we move onto using ShapML for Shapley-based explanations.

In [9]:
#@save "../Models/xANN_Parameters.bson" flux_model

# Loading parameters
@load "../Models/xANN_MSE_v1.bson" best_model
best_model

Chain(
  Dense(129 => 64, relu),               # 8_320 parameters
  Dense(64 => 32, relu),                # 2_080 parameters
  Dense(32 => 7),                       # 231 parameters
)                   # Total: 6 arrays, 10_631 parameters, 41.902 KiB.

In [10]:
std_scaler = MinMaxScaler()
x_train_reduced = std_scaler.fit_transform(x_train)
x_val_reduced = std_scaler.transform(x_val)
x_test_reduced = std_scaler.transform(x_test)

y_test_hat = Array(best_model(x_test_reduced')')

if loss_fn == "MSE"
    reg_Results(loss_fn, y_test_hat, Float64.(hcat(y_test[:, 1:end-1] .* 100.0, y_test[:,end])), lifetimes_test)
else
    reg_Results(loss_fn, y_test_hat, y_test, lifetimes_test)
end

                precision    recall  f1-score   support

MSE: Unhealthy       0.99      0.98      0.98      1865
  MSE: Healthy       0.95      0.97      0.96       871

      accuracy                           0.97      2736
     macro avg       0.97      0.97      0.97      2736
  weighted avg       0.98      0.97      0.97      2736
                     precision    recall  f1-score   support

MSE: No Fan Failure       0.91      0.91      0.91      1772
   MSE: Fan Failure       0.83      0.83      0.83       964

           accuracy                           0.88      2736
          macro avg       0.87      0.87      0.87      2736
       weighted avg       0.88      0.88      0.88      2736
                     precision    recall  f1-score   support

MSE: No LPC Failure       0.93      0.81      0.87      1794
   MSE: LPC Failure       0.71      0.88      0.79       942

           accuracy                           0.84      2736
          macro avg       0.82      0.85      0.

In [11]:
RUL_idxs = sortperm(y_test[:, end])
sorted_RUL_truth = y_test[RUL_idxs, end]
sorted_RUL_preds = y_test_hat[RUL_idxs, end]

layout = Layout(font_family="Sans-Serif", font_color="black", xaxis_title="Test Sample Index", xaxis_title_font_size=20, xaxis_tickfont_size=18,
    yaxis_title="Remaining Useful Life (cycles)", yaxis_title_font_size=20, yaxis_tickfont_size=18,
    width=600, height=400, plot_bgcolor="white", legend_font_size=18, xaxis_linecolor="black",
    xaxis_rangemode="nonnegative", yaxis_linecolor="black", xaxis_tickvals=[0, 500, 1000, 1500, 2000, 2500],
    xaxis_ticks="inside", yaxis_ticks="inside")

trace1 = PlotlyJS.scatter(y=sorted_RUL_preds, mode="markers", marker_symbol="x", marker_size=8,
    marker_line_color="black", name="Prediction", marker_line_width="0.5", marker_color="#3BC8E4")

trace2 = PlotlyJS.scatter(y=sorted_RUL_truth, mode="lines", name="Truth", line_width=4)

p1 = PlotlyJS.Plot([trace1, trace2], layout)

data: [
  "scatter with fields marker, mode, name, type, and y",
  "scatter with fields line, mode, name, type, and y"
]

layout: "layout with fields font, height, legend, margin, plot_bgcolor, template, width, xaxis, and yaxis"

In [12]:
r2score = round(r2_score(sorted_RUL_truth, sorted_RUL_preds), digits=3)

layout = Layout(font_family="Sans-Serif", font_color="black", xaxis_title="Predicted RUL (cycles)", xaxis_title_font_size=20, xaxis_tickfont_size=18,
    yaxis_title="Actual RUL (cycles)", yaxis_title_font_size=20, yaxis_tickfont_size=18,
    width=600, height=400, plot_bgcolor="white", xaxis_linecolor="black", legend_font_size=18,
    xaxis_rangemode="nonnegative", yaxis_linecolor="black", showlegend=true, xaxis_tickvals=[0, 20, 40, 60, 80, 100],
    xaxis_ticks="inside", yaxis_ticks="inside")

trace1 = PlotlyJS.scatter(x=sorted_RUL_preds, y=sorted_RUL_truth, mode="markers", marker_symbol="x", marker_size=8,
    marker_line_color="black", marker_line_width=0.5, marker_color="#3BC8E4", showlegend=false)

trace2 = PlotlyJS.scatter(x=range(0,90), y=range(0,90), mode="lines", line_width=3, line_dash="dash", name="R² = $r2score")

p1 = PlotlyJS.Plot([trace1, trace2], layout)
#PlotlyJS.savefig(p1, "../Plots/RUL_vs_Actual_NASA_Split.pdf",  width=600, height=400, scale=1)

data: [
  "scatter with fields marker, mode, showlegend, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields font, height, legend, margin, plot_bgcolor, showlegend, template, width, xaxis, and yaxis"

In [13]:
delta = y_test_hat - y_test

healthy_idxs = findall(y_test[:, 1] .== 1)
unhealthy_idxs = findall(y_test[:, 1] .== 0)
fan_idxs = findall(y_test[:, 2] .== 1)
lpc_idxs = findall(y_test[:, 3] .== 1)
hpc_idxs = findall(y_test[:, 4] .== 1)
hpt_idxs = findall(y_test[:, 5] .== 1)
lpt_idxs = findall(y_test[:, 6] .== 1)

healthy_errors = delta[healthy_idxs, end]
unhealthy_errors = delta[unhealthy_idxs, end]
fan_errors = delta[fan_idxs, end]
lpc_errors = delta[lpc_idxs, end]
hpc_errors = delta[hpc_idxs, end]
hpt_errors = delta[hpt_idxs, end]
lpt_errors = delta[lpt_idxs, end]

trace1 = box(y=healthy_errors, boxpoints="all", marker_color="indianred", name="Healthy")
trace2 = box(y=unhealthy_errors, boxpoints="all", marker_color="lightseagreen", name="Unhealthy")

layout = Layout(font_family="Sans-Serif", font_color="black", xaxis_title="Health State", xaxis_title_font_size=20, 
    xaxis_tickfont_size=18, yaxis_title="RUL Prediction Error (cycles)", yaxis_title_font_size=20, yaxis_tickfont_size=18,
    showlegend=false, width=600, height=400, plot_bgcolor="white", xaxis_linecolor="black", yaxis_linecolor="black")

p1 = PlotlyJS.Plot([trace1, trace2], layout)
#PlotlyJS.savefig(p1, "../Plots/RUL_HS_NASA_Split.pdf",  width=600, height=400, scale=1)

data: [
  "box with fields boxpoints, marker, name, type, and y",
  "box with fields boxpoints, marker, name, type, and y"
]

layout: "layout with fields font, height, margin, plot_bgcolor, showlegend, template, width, xaxis, and yaxis"

In [14]:
trace_fan = box(y=fan_errors, boxpoints="all", name="Fan")
trace_lpc = box(y=lpc_errors, boxpoints="all", name="LPC")
trace_hpc = box(y=hpc_errors, boxpoints="all", name="HPC")
trace_hpt = box(y=hpt_errors, boxpoints="all", name="HPT")
trace_lpt = box(y=lpt_errors, boxpoints="all", name="LPT")


layout = Layout(font_family="Sans-Serif", font_color="black", xaxis_title="Failing Component",
    xaxis_title_font_size=20, xaxis_tickfont_size=18,
    yaxis_title="RUL Prediction Error (cycles)", yaxis_title_font_size=20, yaxis_tickfont_size=18,
    showlegend=false, width=600, height=400, plot_bgcolor="white", xaxis_linecolor="black", yaxis_linecolor="black")

p1 = PlotlyJS.Plot([trace_fan, trace_lpc, trace_hpc, trace_hpt, trace_lpt], layout)
#PlotlyJS.savefig(p1, "../Plots/RUL_Components_NASA_Split.pdf",  width=600, height=400, scale=1)

data: [
  "box with fields boxpoints, name, type, and y",
  "box with fields boxpoints, name, type, and y",
  "box with fields boxpoints, name, type, and y",
  "box with fields boxpoints, name, type, and y",
  "box with fields boxpoints, name, type, and y"
]

layout: "layout with fields font, height, margin, plot_bgcolor, showlegend, template, width, xaxis, and yaxis"

# ShapML for XAI Clustering
We start by defining helper functions to help with feature names, Shapley value estimation, and plotting that we will be doing for the rest of this notebook.

In [ ]:
function get_feature_names()
    feature_names = ["Total Duration", "Cycle", "Flight Class"]
    
    for i=1:4 
        push!(feature_names, "W_" * string(i) * " Mean")
        push!(feature_names, "W_" * string(i) * " Std")
        push!(feature_names, "W_" * string(i) * " Min")
        push!(feature_names, "W_" * string(i) * " Q1")
        push!(feature_names, "W_" * string(i) * " Median")
        push!(feature_names, "W_" * string(i) * " Q3")
        push!(feature_names, "W_" * string(i) * " Max")
    end
    
     for i=1:14
        push!(feature_names, "Xs_" * string(i) * " Mean")
        push!(feature_names, "Xs_" * string(i) * " Std")
        push!(feature_names, "Xs_" * string(i) * " Min")
        push!(feature_names, "Xs_" * string(i) * " Q1")
        push!(feature_names, "Xs_" * string(i) * " Median")
        push!(feature_names, "Xs_" * string(i) * " Q3")
        push!(feature_names, "Xs_" * string(i) * " Max")
    end
    
    return feature_names
end


function get_SHAP(x, feature_names, model, sample_size; target="RUL", seed=1)
    since = time()
    explain = DataFrame(x, feature_names)
        
    function predict_function_Health(model, data)
        data_pred = DataFrame(y_pred = model(Array(data)')[1, :])
        return data_pred
    end
    
    function predict_function_Fan(model, data)
        data_pred = DataFrame(y_pred = model(Array(data)')[2, :])
        return data_pred
    end
        
    function predict_function_LPC(model, data)
        data_pred = DataFrame(y_pred = model(Array(data)')[3, :])
        return data_pred
    end
    
    function predict_function_HPC(model, data)
        data_pred = DataFrame(y_pred = model(Array(data)')[4, :])
        return data_pred
    end
        
    function predict_function_HPT(model, data)
        data_pred = DataFrame(y_pred = model(Array(data)')[5, :])
        return data_pred
    end
        
    function predict_function_LPT(model, data)
        data_pred = DataFrame(y_pred = model(Array(data)')[6, :])
        return data_pred
    end
        
    function predict_function_RUL(model, data)
        data_pred = DataFrame(y_pred = model(Array(data)')[end, :])
        return data_pred
    end
    #------------------------------------------------------------------------------
    #Compute stochastic Shapley values.
    
    if target == "Health"
        data_shap = ShapML.shap(explain = explain, model = model, 
                            predict_function = predict_function_Health, sample_size = sample_size,seed = seed)
    elseif target == "Fan"
        data_shap = ShapML.shap(explain = explain, model = model,
                            predict_function = predict_function_Fan, sample_size = sample_size,seed = seed)
    
    elseif target == "LPC"
        data_shap = ShapML.shap(explain = explain, model = model,
                            predict_function = predict_function_LPC, sample_size = sample_size,seed = seed)
        
    elseif target == "HPC"
        data_shap = ShapML.shap(explain = explain, model = model, 
                            predict_function = predict_function_HPC, sample_size = sample_size,seed = seed)
        
    elseif target == "HPT"
        data_shap = ShapML.shap(explain = explain, model = model,
                            predict_function = predict_function_HPT, sample_size = sample_size,seed = seed)
        
    elseif target == "LPT"
        data_shap = ShapML.shap(explain = explain, model = model,
                            predict_function = predict_function_LPT, sample_size = sample_size,seed = seed)
        
    else
        data_shap = ShapML.shap(explain = explain, model = model,
                            predict_function = predict_function_RUL, sample_size = sample_size, seed=seed)
    end
    

    #show(data_shap, allcols = true)
    
    elapsed_time = time() - since
    printfmt("Time Elapsed: {:.2f} seconds", elapsed_time)
    return data_shap, explain
end

function mean_SHAP_plot(data_shap, setting=Theme(bar_spacing = 1mm,
        major_label_font_size=16pt, minor_label_font_size=14pt))

    data_shap[!, :shap_effect_max] = data_shap[!, :shap_effect] + (2 .* data_shap[!, :shap_effect_sd] ./ sqrt(sample_size))
    data_shap[!, :shap_effect_min] = data_shap[!, :shap_effect] - (2 .* data_shap[!, :shap_effect_sd] ./ sqrt(sample_size))
    gdf = groupby(data_shap, [:feature_name])

    data_plot = combine(gdf, [:shap_effect, :shap_effect_max, :shap_effect_min] .=> (x -> mean(x)) .=> [:mean_effect, :mean_effect_max, :mean_effect_min])
    data_plot_2 = combine(gdf, [:shap_effect] .=> (x -> mean(abs.(x))) .=> [:mean_abs_effect])

    data_plot[!, :mean_abs_effect] = data_plot_2[!, :mean_abs_effect]
    
    data_plot = sort(data_plot, order(:mean_abs_effect, rev = true))

    baseline = round(data_shap.intercept[1], digits = 1)

    p = Gadfly.plot(data_plot[1:10, :], y = :feature_name, x = :mean_effect, xmin = :mean_effect_min,
             xmax = :mean_effect_max, Coord.cartesian(yflip = true),
             Scale.y_discrete, Geom.errorbar, Geom.bar(position = :dodge, orientation = :horizontal),
             Guide.xlabel("Mean Shapley Effect"), Guide.ylabel(nothing),
             Guide.title("Feature Importance"), setting)
    
end

function mean_abs_SHAP_plot(data_shap, setting=Theme(bar_spacing = 1mm,
    major_label_font_size=16pt, minor_label_font_size=14pt))

    data_shap[!, :shap_effect_max] = data_shap[!, :shap_effect] + (2 .* data_shap[!, :shap_effect_sd] ./ sqrt(sample_size))
    data_shap[!, :shap_effect_min] = data_shap[!, :shap_effect] - (2 .* data_shap[!, :shap_effect_sd] ./ sqrt(sample_size))
    gdf = groupby(data_shap, [:feature_name])

    data_plot = combine(gdf, [:shap_effect, :shap_effect_max, :shap_effect_min] .=> (x -> mean(x)) .=> [:mean_effect, :mean_effect_max, :mean_effect_min])
    data_plot_2 = combine(gdf, [:shap_effect] .=> (x -> mean(abs.(x))) .=> [:mean_abs_effect])

    data_plot[!, :mean_abs_effect] = data_plot_2[!, :mean_abs_effect]
    
    data_plot = sort(data_plot, order(:mean_abs_effect, rev = true))

    baseline = round(data_shap.intercept[1], digits = 1)

    p = Gadfly.plot(data_plot[1:10, :], y = :feature_name, x = :mean_effect, xmin = :mean_effect_min,
             xmax = :mean_effect_max, Coord.cartesian(yflip = true),
             Scale.y_discrete, Geom.errorbar, Geom.bar(position = :dodge, orientation = :horizontal),
             Guide.xlabel("Mean Shapley Effect"), Guide.ylabel(nothing),
             Guide.title("Feature Importance"), setting)
    
end

function stds_SHAP_plot(data_shap, m, setting=Theme(bar_spacing = 1mm))
   
    gdf = groupby(data_shap, [:feature_name])
    data_plot = combine(gdf, [:shap_effect_sd] .=> (x -> mean(x ./ sqrt(m))) .=> [:mean_effect])

    data_plot = sort(data_plot, order(:mean_effect, rev = true))

    baseline = round(data_shap.intercept[1], digits = 1)

    p = Gadfly.plot(data_plot[1:10, :], y = :feature_name, x = :mean_effect, Coord.cartesian(yflip = true),
             Scale.y_discrete, Geom.bar(position = :dodge, orientation = :horizontal),
             Guide.xlabel("Mean Monte Carlo Standard Error"), Guide.ylabel(nothing),
             Guide.title("Shapley Uncertainties"), setting)
    
end

function feature_SHAP_plot(data_shap; feature="Cycle")
    data_plot = data_shap[data_shap.feature_name .== feature, :]  # Selecting 1 feature for ease of plotting.

    baseline = round(data_shap.intercept[1], digits = 1)

    p_points = Gadfly.layer(data_plot, x = :feature_value, y = :shap_effect, Geom.point())
    p_line = Gadfly.layer(data_plot, x = :feature_value, y = :shap_effect, Geom.smooth(method = :loess, smoothing = 0.5),
                   style(line_width = 0.75mm,), Theme(default_color = "black"))
    p = Gadfly.plot(p_line, p_points, Guide.xlabel("Feature value"), Guide.ylabel("Shapley effect (baseline = $baseline)"),
             Guide.title("Feature Effect - $(data_plot.feature_name[1])"), Coord.cartesian(xmin=0, xmax=1))
    
end

In [ ]:
feature_names = get_feature_names()
sample_size = 60  # Number of Monte Carlo samples.

data_shap, explain = get_SHAP(x_test_reduced, feature_names, best_model, sample_size; target="Fan");

In [ ]:
data_shap

In [ ]:
mean_SHAP_plot(data_shap)

In [ ]:
shap_top_feature_names = ["Xs_4 Mean",
"Xs_14 Mean",
"Xs_14 Max",
"Xs_9 Mean",
"Xs_8 Mean",
"Xs_3 Q3",
"Xs_5 Mean",
"Xs_8 Max",
"Xs_2 Q3",
"Cycle"]

mean_abs_shap_effects = [32.79252,
32.64178,
31.0926,
27.94958,
24.1156,
22.04892,
21.91548,
19.45974,
18.58562,
18.19066]

stds_abs_shap_effects = [2.34247807,
1.146650247,
3.029698124,
2.624729599,
2.344547214,
1.053923265,
3.116376541,
1.223164352,
4.535747803,
3.915672776]

setting=Theme(bar_spacing = 1mm,
    major_label_font_size=16pt, minor_label_font_size=14pt)

p = Gadfly.plot(data_plot[1:10, :], y = shap_top_feature_names, x = mean_abs_shap_effects, xmin = mean_abs_shap_effects - 2 .* stds_abs_shap_effects,
             xmax = mean_abs_shap_effects + 2 .* stds_abs_shap_effects, Coord.cartesian(yflip = true),
             Scale.y_discrete, Geom.errorbar, Geom.bar(position = :dodge, orientation = :horizontal),
             Guide.xlabel("Mean Absolute Shapley Effect"), Guide.ylabel(nothing),
             Guide.title("Feature Importance"), setting)

In [ ]:
data_shap[!, :shap_effect_max] = data_shap[!, :shap_effect] + (2 .* data_shap[!, :shap_effect_sd] ./ sqrt(sample_size))
data_shap[!, :shap_effect_min] = data_shap[!, :shap_effect] - (2 .* data_shap[!, :shap_effect_sd] ./ sqrt(sample_size))
gdf = groupby(data_shap, [:feature_name])

data_plot = combine(gdf, [:shap_effect, :shap_effect_max, :shap_effect_min] .=> (x -> mean(x)) .=> [:mean_effect, :mean_effect_max, :mean_effect_min])
data_plot_2 = combine(gdf, [:shap_effect] .=> (x -> mean(abs.(x))) .=> [:mean_abs_effect])

data_plot[!, :mean_abs_effect] = data_plot_2[!, :mean_abs_effect]
    
data_plot = sort(data_plot, order(:mean_abs_effect, rev = true))

In [ ]:
top_features = data_plot[1:10, :feature_name]

top_shap_raw = hcat([select(gdf[(feature_name = feature,)], [:feature_value] .=> feature) for feature in top_features]...)
top_shap = hcat([select(gdf[(feature_name = feature,)], [:shap_effect] .=> feature) for feature in top_features]...)

top_cols = [findfirst(feature .== feature_names ) for feature in top_features]
raw_test_values = x_test[:, top_cols]
raw_test_explain = DataFrame(raw_test_values, top_features)

In [ ]:
import Random
Random.seed!(4)
raw_embedding = umap(Array(x_test_reduced)', 2, n_neighbors=300, min_dist=1e-16)
raw_embedding = raw_embedding'
shap_embedding = umap(Array(top_shap)', 2, n_neighbors=300, min_dist=1e-16)
shap_embedding = shap_embedding'

latex_fonts = Theme(major_label_font="Helvetica", major_label_font_size=16pt,
                    minor_label_font="Helvetica", minor_label_font_size=14pt,
                    key_title_font="Helvetica", key_title_font_size=12pt,
                    key_label_font="Helvetica", key_label_font_size=10pt,
                    grid_line_width=1pt)

Gadfly.push_theme(latex_fonts)

Gadfly.plot(x=raw_embedding[:, 1], y=raw_embedding[:, 2], color=best_model(x_test_reduced')'[:, 1] .> 50, Scale.color_discrete,
             Guide.xlabel("UMAP₁"), Guide.ylabel("UMAP₂"), Guide.colorkey(title="Health State", labels=["Healthy", "Unhealthy"]))

In [ ]:
hdbscan = pyimport("hdbscan")
shap_hdbscan = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=50)

#idxs = findall(best_model(x_test_reduced')'[:, 6] .> 50)  #change as appropriate for failing components
idxs = 1:size(x_test_reduced, 1) #use for RUL and health state preds

# RUL_preds = best_model(x_test_reduced')[end, :]
# RUL_trues = y_test[:, end]

# delta = RUL_preds - RUL_trues

# idxs = abs.(delta) .>= 10

shap_hdbscan.fit(shap_embedding[idxs, :])
Gadfly.plot(x=shap_embedding[idxs, 1], y=shap_embedding[idxs, 2], Geom.point, color=shap_hdbscan.labels_, Scale.color_discrete,
            Guide.xlabel("UMAP₁"), Guide.ylabel("UMAP₂"), Guide.colorkey(title="Cluster"))

In [ ]:
Gadfly.plot(x=shap_embedding[idxs, 1], y=shap_embedding[idxs, 2], Geom.point, color=best_model(x_test_reduced')' .> 50, Scale.color_discrete,
            Guide.xlabel("UMAP₁"), Guide.ylabel("UMAP₂"), Guide.colorkey(title="Health State", labels=["Healthy", "Unhealthy"]))

In [ ]:
Gadfly.plot(x=shap_embedding[idxs, 1], y=shap_embedding[idxs, 2], Geom.point, color=best_model(x_test_reduced')[7, idxs],
            Guide.xlabel("UMAP₁"), Guide.ylabel("UMAP₂"), Guide.colorkey(title="RUL Prediction"))

In [ ]:
var = "Xs_2 Mean"
Gadfly.plot(x=shap_embedding[idxs, 1], y=shap_embedding[idxs, 2], Geom.point, color=raw_test_explain[idxs, var],
            Guide.xlabel("UMAP₁"), Guide.ylabel("UMAP₂"), Guide.colorkey(title=var))

In [ ]:
imodels = pyimport("imodels")
cluster = 1
yc = (shap_hdbscan.labels_ .== cluster) * 1
sr = imodels.SkopeRulesClassifier(max_depth=2, n_estimators=1000, random_state=43).fit(raw_test_values[idxs, :], yc, feature_names=top_features)
sr.rules_

In [ ]:
print(classification_report(yc, sr._predict_top_rules(raw_test_values[idxs, :], 1)))

In [ ]:
(shap_hdbscan.labels_ .== 0) * 1